In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.utils import to_categorical
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from keras import backend as K

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define custom metrics
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    precision_val = precision(y_true, y_pred)
    recall_val = recall(y_true, y_pred)
    return 2*((precision_val*recall_val)/(precision_val+recall_val + K.epsilon()))

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode= "fine")

# Preprocess the data
x_train = preprocess_input(x_train.astype("float32"))
x_test = preprocess_input(x_test.astype("float32"))
y_train = to_categorical(y_train, num_classes= 100)
y_test = to_categorical(y_test, num_classes= 100)

# Parameters
epochs = 30
batch_size = 32
learning_rate = 0.001
momentum = 0.9

# Load pre-trained ResNet50 model
base_model = ResNet50(weights= "imagenet", include_top= False, input_shape= (32, 32, 3))

# Add custom layers on top of pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation= "relu")(x)
predictions = Dense(100, activation= "softmax")(x)

# Combine base model and custom layers into a new model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=SGD(learning_rate=learning_rate, momentum=momentum),
              loss= "categorical_crossentropy",
              metrics=["accuracy", recall, precision, f1])

# Train the model
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

# Plot training and validation accuracy
plt.plot(history.history["accuracy"], label= "Training Accuracy")
plt.plot(history.history["val_accuracy"], label= "Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.show()


Epoch 1/30
1563/1563 [==============================] - 30s 16ms/step - loss: 2.8584 - accuracy: 0.3185 - recall: 0.1759 - precision: 0.5880 - f1: 0.2668 - val_loss: 2.4551 - val_accuracy: 0.3762 - val_recall: 0.2196 - val_precision: 0.6660 - val_f1: 0.3263
Epoch 2/30
1563/1563 [==============================] - 24s 15ms/step - loss: 1.9902 - accuracy: 0.4711 - recall: 0.3027 - precision: 0.7380 - f1: 0.4250 - val_loss: 2.3591 - val_accuracy: 0.3989 - val_recall: 0.2648 - val_precision: 0.6507 - val_f1: 0.3728
Epoch 3/30
1563/1563 [==============================] - 24s 15ms/step - loss: 1.6345 - accuracy: 0.5525 - recall: 0.3808 - precision: 0.7984 - f1: 0.5115 - val_loss: 2.3306 - val_accuracy: 0.4134 - val_recall: 0.2900 - val_precision: 0.6496 - val_f1: 0.3977
Epoch 4/30
1563/1563 [==============================] - 22s 14ms/step - loss: 1.3588 - accuracy: 0.6255 - recall: 0.4534 - precision: 0.8409 - f1: 0.5855 - val_loss: 2.3400 - val_accuracy: 0.4121 - val_recall: 0.3004 - val_pre

# **Epochs: 30    Learning Rate: 0.001     Momentum: 0.9    Batch size: 32**